In [ ]:
pip install streamlit

In [3]:
import os
import json
import datetime
import csv
import nltk
import ssl
import streamlit as st
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [4]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath("nltk_data"))
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [1]:
intents=[
    {"tag":"greeting",
     "patterns":["Hi","Hello","What's up?","How you doing?","sup?","What's good?","Hey","Yo"],
     "responses":["Hi!","Hello There!", "Nothing much really","I'm doing well! what about you?","The sky…haha jk what’s up with you?","Heyya!","Yoo what's poppin?"]},
    {
        "tag":"goodbye",
        "patterns":["Bbyee","See ya later","Goodbye","take care","goodnight","See you soon", "Adios!"],
        "responses":["Byeeeee!","Until next time~","Goodbye, take care <3","Take care, I'll see you soon!","Goodnight! Sleep well","Yup! Don't miss me too much","Adios amigo!"],
    },

    {
      "tag":"thanks",
      "patterns":["Thank you","Thanks","Thanks alot","I appreciate it"],
      "responses":["No problem!","No need to mention it!","You're Welcome","Glad I could help"]
    },
    {
      "tag":"about",
      "patterns":["What are you?","Who are you","What can you do","What is your purpose?"],
      "responses":["I'm an AI Chatbot!","I'm Chatzy, your personal Chatbot","I can chat, I can tell jokes, I can help you learn, I can do anything :D ","My purpose is to assist you. I'm just a pocket buddy you could talk to about anything! My purpose is to help you learn while holding conversations!"]
    },
    {
      "tag":"help",
      "patterns":["help","I need help","Can you help me","What should I do?"],
      "responses":["Sure, what do you need help with?","I'm here to help. What's the problem?","How can I assist you?"]
    },
    {
      "tag":"joke",
      "patterns":["Tell me a joke","Say something funny","Make me laugh","What's your best joke?"],
      "responses":["I’m afraid of elevators, so I take steps to avoid them.","What did one DNA say to the other DNA? “Do these genes make me look fat?","What happens to an illegally parked frog? It gets toad away","Knock, knock. Who’s there? Nobel. Nobel who? No­bel, so I knock knocked."]
    },
    {
      "tag":"age",
      "patterns":["How old are you?","What's your age?","When is you birthday?"],
      "responses":["I never age, I'm a chatbot","I don't have an age I'm a chatbot! What about you?","I don't have a birthday but I was born in the digital world"]
    },
      {
        "tag": "sports",
        "patterns": [
            "Tell me about sports",
            "What's your favorite sport",
            "Any sports news"
        ],
        "responses": [
            "While I don't have personal preferences, sports can be exciting! Keep an eye on sports news websites for the latest updates and scores.",
            "Sports are a great way to stay active and entertained. There's a wide range of sports to explore and enjoy!"
        ]
    },
    {
        "tag": "pets",
        "patterns": [
            "Do you have any pets",
            "Tell me about pets",
            "Do you like animals"
        ],
        "responses": [
            "As an AI, I don't have pets, but I'm interested in animals and can answer your questions about them.",
            "Pets can bring joy and companionship into our lives. Many people love keeping dogs, cats, birds, and more as their pets."
        ]
    },
    {
        "tag": "travel",
        "patterns": [
            "Where is a good place to travel",
            "Recommend me a travel destination",
            "Tell me about your favorite travel spot"
        ],
        "responses": [
            "I'm just a chatbot, so I don't travel, but there are many beautiful destinations around the world to explore. It depends on your preferences, such as beaches, mountains, historical sites, or bustling cities.",
            "Traveling allows you to experience different cultures, try new cuisines, and create lasting memories."
        ]
    },
    {
        "tag": "books",
        "patterns": [
            "What's your favorite book",
            "Recommend me a book to read",
            "Tell me about books"
        ],
        "responses": [
            "As an AI, I don't have personal preferences, but there are countless amazing books in various genres. Some popular ones include Harry Potter, To Kill a Mockingbird, and 1984.",
            "Reading books can broaden your knowledge, enhance creativity, and provide a great way to relax."
        ]
    },
    {
        "tag": "education",
        "patterns": [
            "Tell me about education",
            "What's the importance of education",
            "How to study effectively"
        ],
        "responses": [
            "Education is the process of acquiring knowledge, skills, values, and attitudes. It plays a crucial role in personal and societal development.",
            "Studying effectively involves setting clear goals, creating a conducive study environment, staying organized, taking regular breaks, and seeking help when needed."
        ]
    },
    {
        "tag": "health",
        "patterns": [
            "How to stay healthy",
            "Tell me about health tips",
            "What's the importance of fitness"
        ],
        "responses": [
            "To stay healthy, maintain a balanced diet, engage in regular physical activity, get enough sleep, manage stress, and avoid harmful habits.",
            "Fitness is essential for overall well-being and can improve your mood, energy levels, and longevity."
        ]
    },
    {
        "tag": "coding",
        "patterns": [
            "Tell me about coding",
            "How to start coding",
            "What's the best programming language"
        ],
        "responses": [
            "Coding is the process of writing instructions for computers to execute tasks. It powers software, websites, and apps.",
            "If you want to start coding, choose a programming language like Python, Java, or JavaScript, and explore online tutorials and courses."
        ]
    },
    {
        "tag": "art",
        "patterns": [
            "Tell me about art",
            "What's your favorite artwork",
            "How to appreciate art"
        ],
        "responses": [
            "Art comes in various forms, such as paintings, sculptures, music, literature, and more. It's a way to express emotions and ideas.",
            "Appreciating art involves being open-minded, observing details, understanding the context, and exploring different art movements."
        ]
    },
    {
        "tag": "career",
        "patterns": [
            "How to find a job",
            "Tell me about career development",
            "What's the best career advice"
        ],
        "responses": [
            "To find a job, identify your skills and interests, create a compelling resume, network, and apply to suitable positions.",
            "Career development involves setting goals, continuous learning, seeking mentorship, and adapting to the evolving job market."
        ]
    },
    {
        "tag": "technology_help",
        "patterns": [
            "How can you assist with technology",
            "Tell me about tech support",
            "Can you help with computer issues"
        ],
        "responses": [
            "As an AI chatbot, I can provide information and basic troubleshooting for common technology problems.",
            "For complex technical issues, it's best to consult a qualified IT professional."
        ]
    },
    {
        "tag": "history",
        "patterns": [
            "Tell me about history",
            "What's your favorite historical period",
            "How can I learn about historical events"
        ],
        "responses": [
            "History is the study of past events, societies, and civilizations. It provides insights into our roots and informs our future.",
            "Learning about history can involve reading books, visiting museums, watching documentaries, and attending history lectures."
        ]
    },
    {
        "tag": "music",
        "patterns": [
            "What's your favorite music",
            "Recommend me a song",
            "Tell me about music genres"
        ],
        "responses": [
            "As an AI, I don't have personal preferences, but there are various music genres like pop, rock, classical, hip-hop, and more.",
            "Music can evoke emotions, improve mood, and be a source of inspiration."
        ]
    },
    {
        "tag": "exercise",
        "patterns": [
            "How to stay fit",
            "Tell me about exercise",
            "What's the importance of physical activity"
        ],
        "responses": [
            "Staying fit involves a mix of cardiovascular exercises, strength training, flexibility exercises, and a balanced diet.",
            "Regular exercise can enhance physical health, mental well-being, and boost energy levels."
        ]
    },
    {
        "tag": "mindfulness",
        "patterns": [
            "What is mindfulness",
            "Tell me about mindfulness techniques",
            "How to practice mindfulness"
        ],
        "responses": [
            "Mindfulness is the practice of being fully present in the moment, observing thoughts and feelings without judgment.",
            "Mindfulness techniques include meditation, deep breathing, body scanning, and mindful eating."
        ]
    },
    {
        "tag": "science",
        "patterns": [
            "Tell me about science",
            "What's your favorite branch of science",
            "How does science impact our lives"
        ],
        "responses": [
            "Science is the systematic study of the natural world, aiming to understand phenomena and make discoveries.",
            "Science has led to advancements in medicine, technology, agriculture, and our understanding of the universe."
        ]
    },
    {
        "tag": "gaming",
        "patterns": [
            "Do you play games",
            "Tell me about gaming",
            "What's your favorite video game"
        ],
        "responses": [
            "As an AI, I don't play games, but I can help you with information about various video games and gaming platforms.",
            "Gaming is a popular form of entertainment enjoyed by people of all ages."
        ]
    },
    {
        "tag": "positivity",
        "patterns": [
            "Spread some positivity",
            "Tell me a positive quote",
            "How to stay positive"
        ],
        "responses": [
            "Every day is a new opportunity to make a positive impact. You are capable of great things!",
            "Remember, you are strong, resilient, and capable of overcoming challenges."
        ]
    },
    {
        "tag": "cooking",
        "patterns": [
            "Tell me about cooking",
            "What's your favorite dish",
            "Cooking tips for beginners"
        ],
        "responses": [
            "Cooking is the art of preparing food, and it allows you to experiment with flavors and create delicious meals.",
            "For beginners, start with simple recipes, use fresh ingredients, and don't be afraid to try new techniques."
        ]
    },
    {
        "tag": "relationship",
        "patterns": [
            "How to maintain a healthy relationship",
            "Tell me about love",
            "Relationship advice"
        ],
        "responses": [
            "Communication, trust, and mutual respect are essential for a healthy and fulfilling relationship.",
            "Love is a complex and beautiful emotion that connects people on a deep level."
        ]
    },
    {
        "tag": "nature",
        "patterns": [
            "Tell me about nature",
            "What's your favorite natural wonder",
            "How to protect the environment"
        ],
        "responses": [
            "Nature encompasses the beauty and diversity of the world, including landscapes, wildlife, and ecosystems.",
            "Protecting the environment involves reducing waste, conserving resources, and supporting sustainable practices."
        ]
    },
    {
        "tag": "productivity",
        "patterns": [
            "How to be more productive",
            "Tell me about time management",
            "Productivity tips"
        ],
        "responses": [
            "To be more productive, prioritize tasks, set goals, eliminate distractions, and take regular breaks.",
            "Effective time management can lead to increased efficiency and reduced stress."
        ]
    },
    {
        "tag": "travel_tips",
        "patterns": [
            "Tell me about travel tips",
            "How to pack for a trip",
            "What to consider while traveling"
        ],
        "responses": [
            "Travel tips include packing light, carrying essential documents, researching your destination, and respecting local customs.",
            "While traveling, be open to new experiences, try local cuisines, and be mindful of your surroundings."
        ]
    },
    {
        "tag": "languages",
        "patterns": [
            "Tell me about languages",
            "How to learn a new language",
            "What's the importance of multilingualism"
        ],
        "responses": [
            "Languages are a crucial aspect of human communication and culture, with thousands of languages spoken worldwide.",
            "Learning a new language can expand your horizons, enhance cognitive abilities, and facilitate cross-cultural interactions."
        ]
    },
    {
        "tag": "inspiration",
        "patterns": [
            "I need some inspiration",
            "Tell me an inspiring story",
            "How to stay motivated"
        ],
        "responses": [
            "You are capable of achieving remarkable things. Believe in yourself and never give up!",
            "The journey to success may have challenges, but it's the persistence and determination that lead to extraordinary accomplishments."
        ]
    },
    {
        "tag": "finance_tips",
        "patterns": [
            "Tell me about finance tips",
            "How to save money",
            "Investment advice"
        ],
        "responses": [
            "Finance tips include budgeting, saving a portion of your income, avoiding unnecessary expenses, and investing wisely for the future.",
            "Investing in stocks, mutual funds, or real estate can help grow your wealth over time."
        ]
    },
    {
        "tag": "artificial_intelligence",
        "patterns": [
            "Tell me about artificial intelligence",
            "How does AI work",
            "AI applications"
        ],
        "responses": [
            "Artificial intelligence is the simulation of human intelligence in machines, enabling them to learn and perform tasks.",
            "AI has diverse applications, including voice assistants, self-driving cars, recommendation systems, and medical diagnostics."
        ]
    },
    {
        "tag": "motivation",
        "patterns": [
            "I need some motivation",
            "Tell me an encouraging quote",
            "How to overcome obstacles"
        ],
        "responses": [
            "Believe in yourself. You have the strength to overcome any obstacle and achieve your dreams!",
            "Obstacles are stepping stones to success. Embrace challenges as opportunities for growth and learning."
        ]
    }

]

In [ ]:
vectorizer=TfidfVectorizer()
clf=LogisticRegression(random_state=0,max_iter=10000)

# Preprocess the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

# training the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

LogisticRegression(max_iter=10000, random_state=0)

In [ ]:
def chatbot(input_text):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            return response

counter = 0

In [ ]:
chat_1=chatbot("Tell me a joke!")
print(chat_1)

What did one DNA say to the other DNA? “Do these genes make me look fat?


In [ ]:
def main():
    global counter
    st.title("Implementation of Chatbot using NLP")

    # Create a sidebar menu with options
    menu = ["Home", "Conversation History", "About"]
    choice = st.sidebar.selectbox("Menu", menu)

    # Home Menu
    if choice == "Home":
        st.write("Welcome to the chatbot. Please type a message and press Enter to start the conversation.")

        # Check if the chat_log.csv file exists, and if not, create it with column names
        if not os.path.exists('chat_log.csv'):
            with open('chat_log.csv', 'w', newline='', encoding='utf-8') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerow(['User Input', 'Chatbot Response', 'Timestamp'])

        counter += 1
        user_input = st.text_input("You:", key=f"user_input_{counter}")

        if user_input:

            # Convert the user input to a string
            user_input_str = str(user_input)

            response = chatbot(user_input)
            st.text_area("Chatbot:", value=response, height=120, max_chars=None, key=f"chatbot_response_{counter}")

            # Get the current timestamp
            timestamp = datetime.datetime.now().strftime(f"%Y-%m-%d %H:%M:%S")

            # Save the user input and chatbot response to the chat_log.csv file
            with open('chat_log.csv', 'a', newline='', encoding='utf-8') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerow([user_input_str, response, timestamp])

            if response.lower() in ['goodbye', 'bye']:
                st.write("Thank you for chatting with me. Have a great day!")
                st.stop()

    # Conversation History Menu
    elif choice == "Conversation History":
        # Display the conversation history in a collapsible expander
        st.header("Conversation History")
        # with st.beta_expander("Click to see Conversation History"):
        with open('chat_log.csv', 'r', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile)
            next(csv_reader)  # Skip the header row
            for row in csv_reader:
                st.text(f"User: {row[0]}")
                st.text(f"Chatbot: {row[1]}")
                st.text(f"Timestamp: {row[2]}")
                st.markdown("---")

    elif choice == "About":
        st.write("The goal of this project is to create a chatbot that can understand and respond to user input based on intents. The chatbot is built using Natural Language Processing (NLP) library and Logistic Regression, to extract the intents and entities from user input. The chatbot is built using Streamlit, a Python library for building interactive web applications.")

        st.subheader("Project Overview:")

        st.write("""
        The project is divided into two parts:
        1. NLP techniques and Logistic Regression algorithm is used to train the chatbot on labeled intents and entities.
        2. For building the Chatbot interface, Streamlit web framework is used to build a web-based chatbot interface. The interface allows users to input text and receive responses from the chatbot.
        """)

        st.subheader("Dataset:")

        st.write("""
        The dataset used in this project is a collection of labelled intents and entities. The data is stored in a list.
        - Intents: The intent of the user input (e.g. "greeting", "budget", "about")
        - Entities: The entities extracted from user input (e.g. "Hi", "How do I create a budget?", "What is your purpose?")
        - Text: The user input text.
        """)

        st.subheader("Streamlit Chatbot Interface:")

        st.write("The chatbot interface is built using Streamlit. The interface includes a text input box for users to input their text and a chat window to display the chatbot's responses. The interface uses the trained model to generate responses to user input.")

        st.subheader("Conclusion:")

        st.write("In this project, a chatbot is built that can understand and respond to user input based on intents. The chatbot was trained using NLP and Logistic Regression, and the interface was built using Streamlit. This project can be extended by adding more data, using more sophisticated NLP techniques, deep learning algorithms.")

if __name__ == '__main__':
    main()

2024-12-07 18:47:03.504 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 18:47:03.723 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-12-07 18:47:03.726 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 18:47:03.729 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 18:47:03.731 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 18:47:03.735 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 18:47:03.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-07 18:47:03.739 Session state does not 